## LSSTCam Astrometry learning with GAIA

Craig Lage - 19-May-25

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from lsst.daf.butler import Butler
import lsst.afw.geom as afwGeom
from astropy.coordinates import AltAz, ICRS, EarthLocation, Angle, FK5, SkyCoord
import astropy.units as u
from astroquery.gaia import Gaia
from lsst.geom import SpherePoint,Angle,Extent2I,Box2I,Extent2D,Point2D, Point2I
from lsst.obs.lsst.cameraTransforms import LsstCameraTransforms
from lsst.obs.lsst import LsstCam
from lsst.obs.base import createInitialSkyWcsFromBoresight, createInitialSkyWcs
from astropy.coordinates import EarthLocation
location = EarthLocation.of_site('Rubin:Simonyi')
import lsst.geom as geom
from lsst.summit.utils.plotting import plot
from os import listdir
import healpy as hp
from astropy.table import Table, vstack
from astropy.coordinates import angular_separation
import cv2
import sys

In [ ]:
def MakeShiftedWCS(camera, ref_detector, ref_wcs, img_detector):
    img_bbox = img_detector.getBBox()
    ref_bbox = ref_detector.getBBox()
    lct = LsstCameraTransforms(camera,img_detName)
    fpX, fpY = lct.ccdPixelToFocalMm(ref_bbox.centerX,ref_bbox.centerY, img_detName)
    # The factor of 100 below is 1000 microns/mm / 10 microns/pixel = 100 pixels/mm.
    fpX = fpX * 100 + img_bbox.centerX # Convert to pixels and recenter
    fpY = fpY * 100 + img_bbox.centerY # Convert to pixels and recenter
    cdMatrix = ref_wcs.getCdMatrix()
    for i in range(img_detector.getOrientation().getNQuarter()):
        np.rot90(cdMatrix)
    pixelOrigin = Point2D(img_bbox.centerX, img_bbox.centerY)
    skyOrigin = ref_wcs.pixelToSky(Point2D(fpX, fpY))
    img_wcs = afwGeom.makeSkyWcs(pixelOrigin, skyOrigin, cdMatrix)
    return img_wcs


In [ ]:
def QueryGAIA(wcs, bbox, ra_center, dec_center, magLimit=17.0):
    radius = bbox.centerX * np.sqrt(2.0) * 0.2
    radius = radius * u.arcsec # Enough to cover the bbox    
    cols = ['ra', 'dec', 'phot_g_mean_mag', 'designation']
    # Construct the ADQL query
    query = f"""
    SELECT *
    FROM gaiadr3.gaia_source
    WHERE CONTAINS(POINT({ra_center}, {dec_center}), CIRCLE(ra, dec, {radius.to(u.degree).value})) = 1
    AND phot_g_mean_mag < {magLimit} 
    """
    # Launch the query
    job = Gaia.launch_job_async(query)
    results = job.get_results()
    gaia_table = results.to_pandas()
    gaia_table = gaia_table[cols]
    # using the wcs to locate the stars inside the CCD minus npixedge
    ccdx,ccdy = wcs.skyToPixelArray(gaia_table['ra'],gaia_table['dec'], degrees=True)
    inCCD = bbox.contains(ccdx,ccdy)
    gaia_table['ccdx'] = ccdx
    gaia_table['ccdy'] = ccdy
    gaia_table['inCCD'] = inCCD
    return gaia_table


In [ ]:
def CheckWCSScience(camera, wcs, expId, detector, magLimit):
    detName = detector.getName()
    bbox = detector.getBBox()
    calexp = butler.get('preliminary_visit_image', detector=detector.getId(), visit=expId)
    skyCenter = wcs.pixelToSky(Point2D(bbox.centerX, bbox.centerY))
    ra_center = skyCenter.getRa().asDegrees()
    dec_center = skyCenter.getDec().asDegrees()
    gaia_table = QueryGAIA(wcs, bbox, ra_center, dec_center, magLimit=magLimit)
    #for i in range(len(gaia_table)):
    #    print(gaia_table['designation'][i], ccdx[i], ccdy[i], inCCD[i], gaia_table['phot_g_mean_mag'][i])
    xs = []
    ys = []
    mags = []
    for i in range(len(gaia_table)):
        x = gaia_table['ccdx'][i]
        y = gaia_table['ccdy'][i]
        if gaia_table['inCCD'][i]:
            xs.append(x)
            ys.append(y)
            mags.append(f"{gaia_table['phot_g_mean_mag'][i]:.1f}")
    
    fig = plt.figure(figsize=(16,16))
    
    myPlot = plot(calexp, stretch='ccs', figure=fig)
    ax = myPlot.get_axes()[0]
    ax.scatter(xs, ys\
                ,facecolors='none', edgecolors='g', s=200, lw=2)
    ax.set_title(f"{expId} detector {detector.getId()}")
    return fig


In [ ]:
ampDict = {'Segment00':'C00', 'Segment01':'C01', 'Segment02':'C02', 'Segment03':'C03', 
           'Segment04':'C04', 'Segment05':'C05', 'Segment06':'C06', 'Segment07':'C07', 
           'Segment08':'C17', 'Segment09':'C16', 'Segment10':'C15', 'Segment11':'C14', 
           'Segment12':'C13', 'Segment13':'C12', 'Segment14':'C11', 'Segment15':'C10', }


def CheckWCSGuider(camera, wcs, expId, detector, magLimit):
    detName = detector.getName()
    if expId > 2025050900000:
        detName = detector.getName()
    else:
        print("Swapping SG0, SG1")
        # This is necessary because SG0 and SG1 were swapped for a time
        if detName.split('_')[1] == 'SG0':
            detName = detName.split('_')[0] + '_SG1'
        else:
            detName = detName.split('_')[0] + '_SG0'
    detector = camera[detName]
    bbox = detector.getBBox()
    lct = LsstCameraTransforms(camera,detName)
    background = 0
    dummy = np.zeros([bbox.endY, bbox.endX]) + background
    raw = butler.get("guider_raw", exposure=expId, detector=detector.getId(), instrument='LSSTCam')
    mdata = raw.metadata
    cols = mdata['ROICOLS']
    rows = mdata['ROIROWS']
    llX = mdata['ROICOL']
    llY = mdata['ROIROW']
    urX = llX + mdata['ROICOLS']
    urY = llY + mdata['ROIROWS']
    stamp_ampName = ampDict[mdata['ROISEG']]
    print(stamp_ampName)
    x0, y0 = lct.ampPixelToCcdPixel(llX, llY, stamp_ampName, detName)
    x1, y1 = lct.ampPixelToCcdPixel(urX, urY, stamp_ampName, detName)
    print(llX, llY, x0, y0)
    print(urX, urY, x1, y1)
    x0 = int(x0)
    y0 = int(y0)
    x1 = int(x1)
    y1 = int(y1)
    if x1 < x0:
        x0, x1 = x1, x0

    arr = raw[2].stamp_im.image.array
    print(np.min(arr), np.max(arr), np.median(arr))
    arr = arr - np.median(arr)
    dummy[y0:y1, x0:x1] = arr
    fig = plt.figure(figsize=(16,16))
    
    myPlot = plot(dummy, stretch='ccs', figure=fig)
    ax = myPlot.get_axes()[0]
    #ax.scatter(xs, ys\
    #            ,facecolors='none', edgecolors='g', s=200, lw=2)
    ax.set_title(f"{expId} detector {detector.getId()}")
    ax.set_xlim(x0, x1)
    ax.set_ylim(y0, y1)
    return fig


In [ ]:
lct = LsstCameraTransforms(camera,img_detName)

In [ ]:
lct.ampPixelToCcdPixel?

In [ ]:
butler = Butler('/repo/embargo', collections=['LSSTCam/raw/all', 
                                            'LSSTCam/calib/unbounded', 'LSSTCam/runs/nightlyValidation',
                                              'LSSTCam/runs/nightlyValidation/20250425/w_2025_17/DM-50157',
                                             'LSSTCam/raw/guider'])
instrument = 'LSSTCam'

In [ ]:
expId = 2025072300547

ref_detName = 'R22_S11'
camera = LsstCam.getCamera()
ref_detector = camera[ref_detName]
calexp = butler.get('preliminary_visit_image', detector=ref_detector.getId(), visit=expId)
ref_wcs = calexp.getWcs()
magLimit = 17.0

fig = CheckWCSScience(camera, ref_wcs, expId, ref_detector, magLimit)

In [ ]:
img_detName = 'R33_S22'
img_detector = camera[img_detName]
img_wcs = MakeShiftedWCS(camera, ref_detector, ref_wcs, img_detector)
fig = CheckWCSScience(camera, img_wcs, expId, img_detector, magLimit)

In [ ]:
img_detName = 'R40_SG0'
img_detector = camera[img_detName]
img_wcs = MakeShiftedWCS(camera, ref_detector, ref_wcs, img_detector)
fig = CheckWCSGuider(camera, img_wcs, expId, img_detector, magLimit)

In [ ]:
print(ref_detector.getType().name)
guider_detName = 'R40_SG0'
guider_detector = camera[guider_detName]
print(guider_detector.getType().name)

In [ ]:
expId = 2025072300547

ref_detName = 'R22_S11'
camera = LsstCam.getCamera()
ref_detector = camera[ref_detName]
calexp = butler.get('preliminary_visit_image', detector=ref_detector.getId(), visit=expId)
ref_wcs = calexp.getWcs()
magLimit = 17.0

CheckWCS(camera, ref_wcs, expId, ref_detector, magLimit)

## The cell below is from Shuang.  I have modified it to use focal plane coordinates to allow applying the WCS from one detector to another detector.  This is intended for use with imaging CCDs.

In [ ]:
def get_stars_focal_plane(camera, img_detName, ref_detName, ra, dec, ref_wcs, npixedge=0):
    # some constants and useful objects
    ccd_diag = 0.15852  #Guider CCD diagonal radius in Degrees
    path = '/home/s/shuang92/rubin-user/Monster_guide'  ## RSP path
    res = 5
    nside = 2**res
    npix = 12 * nside**2
    bad_guideramps = {193: 'C1', 198: 'C1', 201: 'C0'}
    filters = ['u','g','r','i','z','y']
    
    # query the Monster
    hp_ind = hp.ang2pix(nside, ra.asDegrees(), dec.asDegrees(), lonlat=True)

    # should only need at most 4 tiles, but for simplicity using 9 Tables
    SW, W, NW, N, NE, E, SE, S = hp.get_all_neighbours(nside, hp_ind)
    
    this_table = Table.read(f'{path}/{hp_ind}.csv')

    E_table = Table.read(f'{path}/{E}.csv')
    W_table = Table.read(f'{path}/{W}.csv')
    S_table = Table.read(f'{path}/{S}.csv')
    N_table = Table.read(f'{path}/{N}.csv')

    SW_table = Table.read(f'{path}/{SW}.csv')
    SE_table = Table.read(f'{path}/{SE}.csv')
    NW_table = Table.read(f'{path}/{NW}.csv')
    NE_table = Table.read(f'{path}/{NE}.csv')

    star_cat = vstack([this_table, E_table, W_table, S_table, N_table, SW_table, SE_table, NW_table, NE_table])

    # find the image detector coordinates in pixels on the focal plane
    img_detector = camera[img_detName]
    img_bbox = img_detector.getBBox()
    nx,ny = img_bbox.getDimensions()
    lct = LsstCameraTransforms(camera,img_detName)
    llfpX, llfpY = lct.ccdPixelToFocalMm(0, 0, img_detName)
    # The factor of 100 below is 1000 microns/mm / 10 microns/pixel = 100 pixels/mm.
    llfpX = llfpX * 100 + int(nx/2) # Convert to pixels and recenter
    llfpY = llfpY * 100 + int(ny/2) # Convert to pixels and recenter
    urfpX, urfpY = lct.ccdPixelToFocalMm(nx, ny, img_detName)
    urfpX = urfpX * 100 + int(nx/2) # Convert to pixels and recenter
    urfpY = urfpY * 100 + int(ny/2) # Convert to pixels and recenter
    if llfpX > urfpX:
        llfpX, urfpX = urfpX, llfpX
    if llfpY > urfpY:
        llfpY, urfpY = urfpY, llfpY

    ref_detector = camera[ref_detName]
    ref_bbox = ref_detector.getBBox()
    ref_llfpX, ref_llfpY = lct.ccdPixelToFocalMm(0, 0, ref_detName)
    ref_llfpX = ref_llfpX * 100 + int(nx/2) # Convert to pixels and recenter
    ref_llfpY = ref_llfpY * 100 + int(ny/2) # Convert to pixels and recenter
  
    ccd_bbox = Box2I(Point2I(llfpX - ref_llfpX, llfpY - ref_llfpY), 
                     Point2I(urfpX - ref_llfpX, urfpY - ref_llfpY))

    star_cat['dangle'] = np.degrees(angular_separation(ra.asRadians(),dec.asRadians(),
                            star_cat['coord_ra'],star_cat['coord_dec']))
    insideCCDradius = (star_cat['dangle']<ccd_diag) # inside the Guider CCD radius

    # Selection 1: the Star is isolated and inside the CCD radius
    cat_select1 = star_cat[(insideCCDradius)]

    # using the wcs to locate the stars inside the CCD minus npixedge
    ccdx,ccdy = ref_wcs.skyToPixelArray(cat_select1['coord_ra'],cat_select1['coord_dec'],degrees=False)
    inCCD = ccd_bbox.contains(ccdx,ccdy)
    # fill with CCD pixel x,y
    cat_select1['ccdx'] = ccdx - llfpX + ref_llfpX
    cat_select1['ccdy'] = ccdy - llfpY + ref_llfpY
    cat_select1['inCCD'] = inCCD

    return [llfpX, llfpY], cat_select1

ampDict = {'Segment00':'C00', 'Segment01':'C01', 'Segment02':'C02', 'Segment03':'C03', 
           'Segment04':'C04', 'Segment05':'C05', 'Segment06':'C06', 'Segment07':'C07', 
           'Segment08':'C17', 'Segment09':'C16', 'Segment10':'C15', 'Segment11':'C14', 
           'Segment12':'C13', 'Segment13':'C12', 'Segment14':'C11', 'Segment15':'C10', }

## The cell below uses GAIA to find stars.  It allows applying the WCS from one detector to another detector.  It is only intended for use in guider CCDs.

In [ ]:
def get_stars_focal_plane_gaia(camera, img_detName, ref_detName, stamp_centerX, stamp_centerY, rows, cols, ref_wcs):
    # find the ra,dec of the CCD center
    img_detector = camera[img_detName]
    img_bbox = img_detector.getBBox()
    nx,ny = img_bbox.getDimensions()
    lct = LsstCameraTransforms(camera,img_detName)
    llfpX, llfpY = lct.ccdPixelToFocalMm(0, 0, img_detName)
    llfpX = llfpX * 100 + int(nx/2) # Convert to pixels and recenter
    llfpY = llfpY * 100 + int(ny/2) # Convert to pixels and recenter
    urfpX, urfpY = lct.ccdPixelToFocalMm(nx, ny, img_detName)
    urfpX = urfpX * 100 + int(nx/2) # Convert to pixels and recenter
    urfpY = urfpY * 100 + int(ny/2) # Convert to pixels and recenter
    if llfpX > urfpX:
        llfpX, urfpX = urfpX, llfpX
    if llfpY > urfpY:
        llfpY, urfpY = urfpY, llfpY

    ref_detector = camera[ref_detName]
    ref_bbox = ref_detector.getBBox()
    ref_llfpX, ref_llfpY = lct.ccdPixelToFocalMm(0, 0, ref_detName)
    ref_llfpX = ref_llfpX * 100 + int(nx/2) # Convert to pixels and recenter
    ref_llfpY = ref_llfpY * 100 + int(ny/2) # Convert to pixels and recenter

    ccd_bbox = Box2I(Point2I(llfpX - ref_llfpX + stamp_centerX - cols/2, llfpY - ref_llfpY + stamp_centerY - rows/2), 
                     Point2I(llfpX - ref_llfpX + stamp_centerX + cols/2, llfpY - ref_llfpY + stamp_centerY + rows/2))

    Gaia.MAIN_GAIA_TABLE = "gaiadr3.gaia_source" 

    
    ra_stamp,dec_stamp = ref_wcs.pixelToSky(stamp_centerX - ref_llfpX + llfpX, stamp_centerY - ref_llfpY + llfpY)
    print("In Gaia", stamp_centerX - ref_llfpX + llfpX, stamp_centerY - ref_llfpY + llfpY)
    rad = u.Quantity(0.16, u.deg) # 0.04 degrees is enough to cover a 400x400 stamp
    # Could be reduced for smaller stamps.
    cols = ['ra', 'dec', 'phot_g_mean_mag']
    
    coord = SkyCoord(ra=ra_stamp.asDegrees(), dec=dec_stamp.asDegrees(), unit=(u.degree, u.degree), frame='icrs')
    Gaia.ROW_LIMIT = 500
    r = Gaia.query_object_async(coordinate=coord, radius=rad, columns=cols)
    
    gaia_table = r.to_pandas()
    # using the wcs to locate the stars inside the CCD minus npixedge
    ccdx,ccdy = ref_wcs.skyToPixelArray(gaia_table['ra'],gaia_table['dec'], degrees=True)
    inCCD = ccd_bbox.contains(ccdx,ccdy)

    # fill with CCD pixel x,y
   
    gaia_table['ccdx'] = ccdx - llfpX + ref_llfpX
    gaia_table['ccdy'] = ccdy - llfpY + ref_llfpY
    gaia_table['inCCD'] = inCCD

    return gaia_table


## This cell builds a blank guider CCD image with amps marked, and then adds the stamp.  The final image is in focal plane orientation.

In [ ]:
def make_guider_image(detector, raw):
    detName = detector.getName()
    det_bbox = detector.getBBox()
    lct = LsstCameraTransforms(camera,detector.getName())
    background = 30
    dummy = np.zeros([det_bbox.endY, det_bbox.endX]) + background
    # Make gridlines and label the amps
    line_level = 100
    colour = (255, 255, 255)
    
    font = cv2.FONT_HERSHEY_SIMPLEX
    scale = 3
    thickness = 10
    for amp in detector:
        ampName = amp.getName()
        amp_bbox = amp.getBBox()
        x0, y0 = amp_bbox.beginX, amp_bbox.beginY
        x1, y1 = amp_bbox.endX, amp_bbox.endY
        if x0 > x1:
            x0, x1 = x1, x0
        if y0 > y1:
            y0, y1 = y1, y0
        dummy[y0:y1, x0:x1] = line_level
        amp_bbox.grow(-4)
        x0, y0 = amp_bbox.beginX, amp_bbox.beginY
        x1, y1 = amp_bbox.endX, amp_bbox.endY
        if x0 > x1:
            x0, x1 = x1, x0
        if y0 > y1:
            y0, y1 = y1, y0
        dummy[y0:y1, x0:x1] = background
        origin = (int((x0+x1)/2)-80, y1-100)
        # Label the amp
        cv2.putText(dummy, ampName, origin, font, scale, colour, thickness, bottomLeftOrigin=True)

    # Add the stamp image
    mdata = raw.metadata
    s_llX = mdata['ROICOL']
    s_llY = mdata['ROIROW']
    dx = mdata['ROICOLS']
    dy = mdata['ROIROWS']
    stamp_ampName = ampDict[mdata['ROISEG']]
    x0, y0 = lct.ampPixelToCcdPixel(s_llX, s_llY, stamp_ampName, detName)
    x0 = int(x0)
    y0 = int(y0)
    x1 = x0 - dx
    y1 = y0 + dy
    if x0 > x1:
        x0, x1 = x1, x0
    if y0 > y1:
        y0, y1 = y1, y0

    arr = raw[2].stamp_im.image.array
    arr = arr - np.median(arr)
    flip_arr = np.fliplr(arr)
    dummy[y0:y1, x0:x1] = flip_arr
    dummy_center = [(x0 + x1) / 2, (y0 + y1) / 2]
    
    # Rotate stamp and dummy_center into rotated coord system
    dumx, dumy = (x0 + x1) / 2, (y0 + y1) / 2
    k = detector.getOrientation().getNQuarter()
    for ii in range(k):
        dummy = np.rot90(dummy, k=1, axes=(1,0))
        dumxp = -dumy
        if dumxp > dummy.shape[1]:
            dumxp = dummy.shape[1] - dumxp
        if dumxp < 0:
            dumxp = dummy.shape[1] + dumxp
        dumyp = dumx
        if dumyp > dummy.shape[0]:
            dumyp = dummy.shape[0] - dumyp
        if dumyp < 0:
            dumyp = dummy.shape[0] + dumyp
        dumx = dumxp
        dumy = dumyp
    dummy_center = [dumx, dumy]
    return dummy, dummy_center

def guider_ccd_offsets(img_detName):
    if img_detName == 'R44_SG0':
        fudgeX = -9; fudgeY = -5
    elif img_detName == 'R44_SG1':
        fudgeX = -5; fudgeY = 1
    elif img_detName == 'R40_SG0':
        fudgeX = 1; fudgeY = -6
    elif img_detName == 'R40_SG1':
        fudgeX = 2; fudgeY = -2
    elif img_detName == 'R04_SG0':
        fudgeX = -3; fudgeY = -1
    elif img_detName == 'R04_SG1':
        fudgeX = 0; fudgeY = 4
    elif img_detName == 'R00_SG0':
        fudgeX = 12; fudgeY = 0
    elif img_detName == 'R00_SG1':
        fudgeX = 2; fudgeY = 2
    return fudgeX, fudgeY


## The cell below plots a guider stamp.  The stamp is in the proper orientation as seen in the focal plane.  Star locations from GAIA are added.

In [ ]:
def plot_stamp(camera, expId, ref_detName, ref_wcs, img_detName, ax, mag_limit=18, show_whole_ccd=False):

    img_detector = camera[img_detName]
    img_bbox = img_detector.getBBox()
    img_center = ref_wcs.pixelToSky(img_bbox.centerX, img_bbox.centerY)
    img_ra_center = img_center.getRa().asDegrees()
    img_dec_center = img_center.getDec().asDegrees()
    
    if expId > 2025050900000:
        raw_detName = img_detName
    else:
        print("Swapping SG0, SG1")
        # This is necessary because SG0 and SG1 were swapped for a time
        if img_detName.split('_')[1] == 'SG0':
            raw_detName = img_detName.split('_')[0] + '_SG1'
        if img_detName.split('_')[1] == 'SG1':
            raw_detName = img_detName.split('_')[0] + '_SG0'
    raw_detector = camera[raw_detName]

    raw = butler.get("guider_raw", exposure=expId, detector=raw_detector.getId(), instrument='LSSTCam')
    mdata = raw.metadata
    cols = mdata['ROICOLS']
    rows = mdata['ROIROWS']
    
    dummy, dummy_center = make_guider_image(img_detector, raw)
    print(img_detName)
    cat_select = get_stars_focal_plane_gaia(camera, img_detName, ref_detName, dummy_center[0], dummy_center[1], rows, cols, ref_wcs)
    xs = []
    ys = []
    mags = []
    
    for i in range(len(cat_select)):
        x = cat_select['ccdx'][i]
        y = cat_select['ccdy'][i]
        if cat_select['inCCD'][i] and cat_select['phot_g_mean_mag'][i] < mag_limit:
            xs.append(x)
            ys.append(y)
            mags.append(f"{cat_select['phot_g_mean_mag'][i]:.1f}")
        
    
    fudgeX, fudgeY = guider_ccd_offsets(img_detName)
    ax.set_title(f"{img_detName}")#, WCS from {ref_detName}  FudgeX={fudgeX} pixels, FudgeY={fudgeY} pixels.", fontsize=12)
    img = ax.imshow(dummy,  interpolation='Nearest', cmap='gray', vmin=0, vmax=100, origin='lower')
    ax.scatter(np.array(xs) + fudgeX, np.array(ys) + fudgeY \
                ,facecolors='none', edgecolors='g', s=200, lw=2)
    for x, y, mag in zip(xs, ys, mags):
        ax.text(x + fudgeX + 7,y + fudgeY, mag, color='g')
    if not show_whole_ccd:
        ax.set_xlim(dummy_center[0] - cols/2, dummy_center[0] + cols/2)
        ax.set_ylim(dummy_center[1] - rows/2, dummy_center[1] + rows/2)
    #ax.scatter(dummy_center[0], dummy_center[1], marker='x', s=100, color='red')
    return 

In [ ]:
expId = 2025042500591
#expId = 2025050200787
#expId = 2025050200769
#expId = 2025050200713

ref_detName = 'R22_S11'
img_detName = 'R33_S22'

camera = LsstCam.getCamera()
ref_detector = camera[ref_detName]
calexp = butler.get('preliminary_visit_image', detector=ref_detector.getId(), visit=expId)
ref_wcs = calexp.getWcs()
raw = butler.get('raw', detector=ref_detector.getId(), exposure=expId)
raw_wcs = raw.getWcs()

In [ ]:
import lsst.afw.geom as afwGeom

In [ ]:
afwGeom.makeSkyWcs?

In [ ]:
cd = ref_wcs.getCdMatrix()

In [ ]:
po = ref_wcs.getPixelOrigin()

In [ ]:
so = raw_wcs.getSkyOrigin()

In [ ]:
ref_wcs.getRelativeRotationToWcs(raw_wcs)

In [ ]:
test_wcs = afwGeom.makeSkyWcs(po, so, cd)

In [ ]:
ref_wcs.getRelativeRotationToWcs(test_wcs)

In [ ]:
rot_cd = np.rot90(cd)

In [ ]:
test_rot_wcs = afwGeom.makeSkyWcs(po, so, rot_cd)

In [ ]:
ref_wcs.getRelativeRotationToWcs(test_rot_wcs)

In [ ]:
ref_wcs.copyAtShiftedPixelOrigin?

In [ ]:
text_ref = ref_wcs.writeString()
text_raw = raw_wcs.writeString()
import difflib
diff = difflib.ndiff(text_ref, text_raw)
for line in diff:
    print(line)

## The cell below takes the WCS from one imaging CCD and applies it to another imaging CCD.  Does not work for guider CCDs

In [ ]:
expId = 2025042500591
#expId = 2025050200787
#expId = 2025050200769
#expId = 2025050200713

ref_detName = 'R22_S11'
img_detName = 'R33_S22'

camera = LsstCam.getCamera()
ref_detector = camera[ref_detName]
ref_calexp = butler.get('preliminary_visit_image', detector=ref_detector.getId(), visit=expId)
ref_wcs = ref_calexp.getWcs()
ref_bbox = ref_detector.getBBox()
ref_center = ref_wcs.pixelToSky(ref_bbox.centerX, ref_bbox.centerY)
ref_ra_center = ref_center.getRa().asDegrees()
ref_dec_center = ref_center.getDec().asDegrees()


img_detector = camera[img_detName]
img_calexp = butler.get('preliminary_visit_image', detector=img_detector.getId(), visit=expId)
img_wcs = img_calexp.getWcs()
img_bbox = img_detector.getBBox()
img_center = img_wcs.pixelToSky(img_bbox.centerX, img_bbox.centerY)
img_ra_center = img_center.getRa()
img_dec_center = img_center.getDec()

[llfpX, llfpY], cat_select = get_stars_focal_plane(camera, img_detName, ref_detName, img_ra_center, img_dec_center, ref_wcs)

xs = []
ys = []
mags = []
for i in range(len(cat_select)):
    x = cat_select['ccdx'][i]
    y = cat_select['ccdy'][i]
    if cat_select['inCCD'][i]:
        xs.append(x)
        ys.append(y)
        mags.append(f"{cat_select['mag_r'][i]:.1f}")




# Now plot the data with matplotlib
%matplotlib inline
def colorbar(mappable):
    from mpl_toolkits.axes_grid1 import make_axes_locatable
    last_axes = plt.gca()
    ax = mappable.axes
    fig = ax.figure
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.05)
    cbar = fig.colorbar(mappable, cax=cax)
    plt.sca(last_axes)
    return cbar

plt.figure(figsize=(16,16))
plt.title(f"{expId} {img_detName}, WCS from {ref_detName}")
img = plt.imshow(img_calexp.image.array,  interpolation='Nearest', cmap='gray', vmin=0, vmax=1000, origin='lower')
plt.scatter(xs, ys\
            ,facecolors='none', edgecolors='g', s=200, lw=2)
for x, y, mag in zip(xs, ys, mags):
    plt.text(x+50,y, mag, color='g')
colorbar(img)
plt.savefig(f"/home/c/cslage/u/LSSTCam/images/WCS_{expId}_{img_detName}_WCS_From_{ref_detName}.png")

## Now look at the guider CCDs. This cell plots a single guider stamp, with WCS extrapolated from an imaging CCD with an astrometric solution.  The keyword 'show_whole_ccd' will show the location of the stamp in the CCD if True.

In [ ]:
camera = LsstCam.getCamera()
ref_detName = 'R22_S11'
expId = 2025050200713
img_detName = 'R44_SG0'
ref_detector = camera[ref_detName]
ref_calexp = butler.get('preliminary_visit_image', detector=ref_detector.getId(), visit=expId)
ref_wcs = ref_calexp.getWcs()

fig, ax = plt.subplots(1,1,figsize=(10,10))
plot_stamp(camera, expId, ref_detName, ref_wcs, img_detName, ax, mag_limit=16.5, show_whole_ccd=True)

## This cell plots all 8 guider mode stamps for a given expId.

In [ ]:
# This config object just tells where to put each plot in the combined plot
config = [['R00_SG0', 3, 1], ['R00_SG1', 2, 0], 
          ['R04_SG0', 2, 3], ['R04_SG1', 3, 2], 
          ['R40_SG0', 1, 0], ['R40_SG1', 0, 1],
          ['R44_SG0', 0, 2], ['R44_SG1', 1, 3]]

#expId = 2025042500591
#expId = 2025050200713
#expId = 2025051200508
expId = 2025050500523
ref_detName = 'R22_S11'

camera = LsstCam.getCamera()
ref_detector = camera[ref_detName]
ref_calexp = butler.get('preliminary_visit_image', detector=ref_detector.getId(), visit=expId)
ref_wcs = ref_calexp.getWcs()

fig, axs = plt.subplots(4,4, figsize=(16,16))
plt.suptitle(f"{expId} Guider images", fontsize=24)
plt.subplots_adjust(wspace=0.1, hspace=0.1)
# Clear the axes and plot frames
for i in range(4):
    for j in range(4):
        axs[i][j].axis('off')
        axs[i][j].set_xticks([])
        axs[i][j].set_yticks([])
for [img_detName, i, j] in config:
    plot_stamp(camera, expId, ref_detName, ref_wcs, img_detName, axs[i][j], mag_limit=17.5)
plt.savefig(f"/home/c/cslage/u/Guider_Mode/images/Guiders_{expId}_WCS_From_{ref_detName}_Gaia.png")

## The stuff below all seems to be working.  I need to transfer to another notebook.
## Still need to rotate the rotated CCDs somewhere.

In [ ]:
expId = 2025050200713
#expId = 2025052200483
img_detName = 'R44_SG0'
ref_detName = 'R22_S11'

camera = LsstCam.getCamera()
ref_detector = camera[ref_detName]
ref_raw = butler.get('raw', detector=ref_detector.getId(), exposure=expId, instrument=instrument)
ref_calexp = butler.get('preliminary_visit_image', detector=ref_detector.getId(), visit=expId, 
                        instrument=instrument)
ref_wcs = ref_calexp.getWcs()
raw_wcs = ref_raw.getWcs()

img_detector = camera[img_detName]
img_bbox = img_detector.getBBox()



In [ ]:
from astropy.time import Time
visinfo = ref_raw.visitInfo
date = visinfo.getDate()
print('Date: ',date)
mjd = date.toAstropy()
print('MJD: ',mjd)
obstime = Time(mjd, format='mjd', scale='tai')
print(obstime)

In [ ]:
test=visinfo.getBoresightRaDec()

In [ ]:
test.getRa().asDegrees()

In [ ]:
def shift_boresight(boresight, ref_wcs, raw_wcs):
    rawSkyCenter = raw_wcs.getSkyOrigin()
    refSkyCenter = ref_wcs.pixelToSky(raw_wcs.getPixelOrigin())
    #print(ra, dec)
    deltaRa = rawSkyCenter.getRa().asDegrees() - refSkyCenter.getRa().asDegrees()
    deltaDec = rawSkyCenter.getDec().asDegrees() - refSkyCenter.getDec().asDegrees()
    print(deltaRa*3600, deltaDec*3600)
    boresightRa = boresight.getRa().asDegrees()
    boresightDec = boresight.getDec().asDegrees()
    shiftedBoresight = SpherePoint((boresightRa - deltaRa) * geom.degrees, 
                        (boresightDec - deltaDec) * geom.degrees)
    return shiftedBoresight


In [ ]:
shift_boresight(visinfo.getBoresightRaDec(), ref_wcs, raw_wcs)

In [ ]:
def radec_to_azalt(ra, dec, location, obstime):
    #Convert RA and dec to Azimuth (deg) and Altitude (deg)    
    radec_coord = SkyCoord(ra=ra*u.deg, dec=dec*u.deg, obstime=obstime, location=location, frame='icrs')
    return radec_coord.altaz.az.deg, radec_coord.altaz.alt.deg

In [ ]:
def make_lsstcam_WCS(camera,visitInfo,location,obstime,
                     extra_rotation=Angle(-90.,geom.degrees), 
                     shiftedBoresight=None):
    """
    Parameters
    ----------
    camera : lsst.afw.cameraGeom.Camera 
        Camera object
    visitInfo : lsst.afw.image.VisitInfo 
        visit info from an Image
    location: astropy.coordinates.earth.EarthLocation
        Observatory location
    obstime: astropy.time.core.Time
        Time of Observation
    extra_rotation : lsst.geom.Angle
        Added rotation to add to visitInfo.orientation to align the LSSTCam properly

    Returns
    -------
    cam_wcs : dictionary 
        WCS for each detector, keyed by detector Id
    cam_radec : dictionary 
        [RA,dec] at the center of each detector, keyed by detector Id    
    cam_azalt : dictionary 
        [Az,Alt] at the center of each detector, keyed by detector Id       
    """
    orientation = visitInfo.getBoresightRotAngle()
    if not shiftedBoresight:
        boresight = visitInfo.getBoresightRaDec()
    else: boresight=shiftedBoresight

    orientation_corrected = orientation+extra_rotation
    orientation_corrwrap = orientation_corrected.wrap()   # guessing that this is needed

    # Get WCS and ra,dec and alt,az for each CCD in camera
    cam_wcs = {}
    cam_radec = {}
    cam_azalt = {}

    for det in camera:
        # get WCS
        cam_wcs[det.getId()] = createInitialSkyWcsFromBoresight(boresight, orientation_corrwrap, det, flipX=False)
        # get central pixel
        x0,y0 = det.getBBox().getCenterX(),det.getBBox().getCenterY()  

        # get ra,dec
        ra1,dec1 = cam_wcs[det.getId()].pixelToSky(x0,y0)
        cam_radec[det.getId()] = [ra1.asDegrees(),dec1.asDegrees()]
        cam_azalt[det.getId()] = radec_to_azalt(ra1.asDegrees(),dec1.asDegrees(),
                                                location,obstime)

    return cam_wcs,cam_radec,cam_azalt
    

In [ ]:
def get_stars_focal_plane_gaia_mod(stamp_centerX, stamp_centerY, rows, cols, ref_wcs):

    ccd_bbox = Box2I(Point2I(stamp_centerX - cols/2, stamp_centerY - rows/2), 
                     Point2I(stamp_centerX + cols/2, stamp_centerY + rows/2))

    Gaia.MAIN_GAIA_TABLE = "gaiadr3.gaia_source" 

    ra_stamp,dec_stamp = ref_wcs.pixelToSky(stamp_centerX, stamp_centerY)
    print("In Gaia Mod", stamp_centerX, stamp_centerY)
    print("In Gaia Mod", ra_stamp.asDegrees(), dec_stamp.asDegrees())
    rad = u.Quantity(0.16, u.deg) # 0.04 degrees is enough to cover a 400x400 stamp
    # Could be reduced for smaller stamps.
    cols = ['ra', 'dec', 'phot_g_mean_mag']
    
    coord = SkyCoord(ra=ra_stamp.asDegrees(), dec=dec_stamp.asDegrees(), unit=(u.degree, u.degree), frame='icrs')
    Gaia.ROW_LIMIT = 2000
    r = Gaia.query_object_async(coordinate=coord, radius=rad, columns=cols)
    
    gaia_table = r.to_pandas()
    # using the wcs to locate the stars inside the CCD minus npixedge
    ccdx,ccdy = ref_wcs.skyToPixelArray(gaia_table['ra'],gaia_table['dec'], degrees=True)
    inCCD = ccd_bbox.contains(ccdx,ccdy)

    # fill with CCD pixel x,y
   
    gaia_table['ccdx'] = ccdx
    gaia_table['ccdy'] = ccdy
    gaia_table['inCCD'] = inCCD

    return gaia_table


In [ ]:
def make_guider_image_mod(detector, raw):
    detName = detector.getName()
    det_bbox = detector.getBBox()
    lct = LsstCameraTransforms(camera,detector.getName())
    background = 30
    dummy = np.zeros([det_bbox.endY, det_bbox.endX]) + background
    # Make gridlines and label the amps
    line_level = 100
    colour = (255, 255, 255)
    
    font = cv2.FONT_HERSHEY_SIMPLEX
    scale = 3
    thickness = 10
    for amp in detector:
        ampName = amp.getName()
        amp_bbox = amp.getBBox()
        x0, y0 = amp_bbox.beginX, amp_bbox.beginY
        x1, y1 = amp_bbox.endX, amp_bbox.endY
        if x0 > x1:
            x0, x1 = x1, x0
        if y0 > y1:
            y0, y1 = y1, y0
        dummy[y0:y1, x0:x1] = line_level
        amp_bbox.grow(-4)
        x0, y0 = amp_bbox.beginX, amp_bbox.beginY
        x1, y1 = amp_bbox.endX, amp_bbox.endY
        if x0 > x1:
            x0, x1 = x1, x0
        if y0 > y1:
            y0, y1 = y1, y0
        dummy[y0:y1, x0:x1] = background
        origin = (int((x0+x1)/2)-80, y1-100)
        # Label the amp
        cv2.putText(dummy, ampName, origin, font, scale, colour, thickness, bottomLeftOrigin=True)

    # Add the stamp image
    mdata = raw.metadata
    s_llX = mdata['ROICOL']
    s_llY = mdata['ROIROW']
    dx = mdata['ROICOLS']
    dy = mdata['ROIROWS']
    stamp_ampName = ampDict[mdata['ROISEG']]
    x0, y0 = lct.ampPixelToCcdPixel(s_llX, s_llY, stamp_ampName, detName)
    x0 = int(x0)
    y0 = int(y0)
    x1 = x0 - dx
    y1 = y0 + dy
    if x0 > x1:
        x0, x1 = x1, x0
    if y0 > y1:
        y0, y1 = y1, y0

    arr = raw[2].stamp_im.image.array
    arr = arr - np.median(arr)
    flip_arr = np.fliplr(arr)
    dummy[y0:y1, x0:x1] = flip_arr
    dummy_center = [(x0 + x1) / 2, (y0 + y1) / 2]

    return dummy, dummy_center


In [ ]:
def plot_stamp_mod(camera, expId, ref_detName, ref_wcs, img_detName, ax, mag_limit=18, show_whole_ccd=False):

    img_detector = camera[img_detName]
    img_bbox = img_detector.getBBox()
    img_center = ref_wcs.pixelToSky(img_bbox.centerX, img_bbox.centerY)
    img_ra_center = img_center.getRa().asDegrees()
    img_dec_center = img_center.getDec().asDegrees()

    if expId > 2025050900000:
        raw_detName = img_detName
    else:
        print("Swapping SG0, SG1")
        # This is necessary because SG0 and SG1 were swapped for a time
        if img_detName.split('_')[1] == 'SG0':
            raw_detName = img_detName.split('_')[0] + '_SG1'
        if img_detName.split('_')[1] == 'SG1':
            raw_detName = img_detName.split('_')[0] + '_SG0'
    raw_detector = camera[raw_detName]

    raw = butler.get("guider_raw", exposure=expId, detector=raw_detector.getId(), instrument='LSSTCam')
    mdata = raw.metadata
    cols = mdata['ROICOLS']
    rows = mdata['ROIROWS']
    
    dummy, dummy_center = make_guider_image_mod(img_detector, raw)
    print(img_detName)
    print(dummy_center)
    cat_select = get_stars_focal_plane_gaia_mod(dummy_center[0], dummy_center[1], rows, cols, ref_wcs)
    xs = []
    ys = []
    mags = []
    fudgeX, fudgeY = 0, 0#guider_ccd_offsets(img_detName)    
    for i in range(len(cat_select)):
        x = cat_select['ccdx'][i] + fudgeX
        y = cat_select['ccdy'][i] + fudgeY
        #if cat_select['inCCD'][i] and cat_select['phot_g_mean_mag'][i] < mag_limit:
        if cat_select['phot_g_mean_mag'][i] < mag_limit:
            if x > dummy_center[0] - cols/2 and x < dummy_center[0] + cols/2:
                if y > dummy_center[1] - rows/2 and y < dummy_center[1] + rows/2:
                    xs.append(x)
                    ys.append(y)
                    mags.append(f"{cat_select['phot_g_mean_mag'][i]:.1f}")
                
    # Rotate stamp image and gaia stars into rotated coord system
    k = img_detector.getOrientation().getNQuarter()
    dumx, dumy = dummy_center[0], dummy_center[1]
    for ii in range(k):
        dummy = np.rot90(dummy, k=1, axes=(1,0))
        dumxp = -dumy
        if dumxp > dummy.shape[1]:
            dumxp = dummy.shape[1] - dumxp
        if dumxp < 0:
            dumxp = dummy.shape[1] + dumxp
        dumyp = dumx
        if dumyp > dummy.shape[0]:
            dumyp = dummy.shape[0] - dumyp
        if dumyp < 0:
            dumyp = dummy.shape[0] + dumyp
        dumx = dumxp
        dumy = dumyp
        dummy_center = [dumx, dumy]

        for nn in range(len(xs)):
            xp = -ys[nn]
            if xp > dummy.shape[1]:
                xp = dummy.shape[1] - xp
            if xp < 0:
                xp = dummy.shape[1] + xp
            yp = xs[nn]
            if yp > dummy.shape[0]:
                yp = dummy.shape[0] - yp
            if yp < 0:
                yp = dummy.shape[0] + yp
            xs[nn] = xp
            ys[nn] = yp
          
    ax.set_title(f"{img_detName} Initial WCS FudgeX={fudgeX} pixels, FudgeY={fudgeY} pixels.", fontsize=12)
    img = ax.imshow(dummy,  interpolation='Nearest', cmap='gray', vmin=0, vmax=100, origin='lower')
    ax.scatter(np.array(xs), np.array(ys) \
                ,facecolors='none', edgecolors='g', s=200, lw=2)
    for x, y, mag in zip(xs, ys, mags):
        ax.text(x + 7,y, mag, color='g')
    if not show_whole_ccd:
        ax.set_xlim(dummy_center[0] - cols/2, dummy_center[0] + cols/2)
        ax.set_ylim(dummy_center[1] - rows/2, dummy_center[1] + rows/2)
    #ax.scatter(dummy_center[0], dummy_center[1], marker='x', s=100, color='red')
    return 

def guider_ccd_offsets(img_detName):
    if img_detName == 'R44_SG0':
        #fudgeX = 112; fudgeY = -382
        fudgeX = 23; fudgeY = -100
    elif img_detName == 'R44_SG1':
        fudgeX = 0; fudgeY = 0
        #fudgeX = -377; fudgeY = -110
        #fudgeX = -97; fudgeY = -18
    elif img_detName == 'R40_SG0':
        fudgeX = -390; fudgeY = -113
        #fudgeX = -87; fudgeY = -57
    elif img_detName == 'R40_SG1':
        #fudgeX = -57; fudgeY = 87
        fudgeX = -113; fudgeY = 390
    elif img_detName == 'R04_SG0':
        fudgeX = -3; fudgeY = -1
    elif img_detName == 'R04_SG1':
        fudgeX = 0; fudgeY = 4
    elif img_detName == 'R00_SG0':
        fudgeX = 12; fudgeY = 0
    elif img_detName == 'R00_SG1':
        fudgeX = 2; fudgeY = 2
    return fudgeX, fudgeY
    

In [ ]:
shiftedBoresight = shift_boresight(visinfo.getBoresightRaDec(), ref_wcs, raw_wcs)

cam_wcs,cam_radec,cam_azalt = \
make_lsstcam_WCS(camera,ref_raw.visitInfo,location,obstime,
                 extra_rotation=Angle(0.,geom.degrees),
                shiftedBoresight=shiftedBoresight)

In [ ]:
cam_wcs[201]

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(10,10))
plot_stamp_mod(camera, expId, ref_detName, cam_wcs[img_detector.getId()], img_detName, 
                ax, mag_limit=18.5, show_whole_ccd=False)
plt.savefig(f"/home/c/cslage/u/Guider_Mode/images/Guiders_{expId}_{img_detName}_Initial_WCS_Gaia.png")

In [ ]:
ref_wcs.getRelativeRotationToWcs(cam_wcs[img_detector.getId()])

In [ ]:
ax.

In [ ]:
#raw_wcs ra/dec = 255.56262818377436 -40.53714226851448
#raw_wcs dummy_center = [3815.0, 1000.0]
#In Gaia 29700.0 22225.0

#caw_wcs ra/dec = 253.20141100595663 -40.16564447917712
#can_wcs dummy_center = [3815.0, 1000.0]
#In Gaia 29700.0 22225.0

In [ ]:
createInitialSkyWcsFromBoresight?

In [ ]:
ref_wcs.